In [1]:
import geopandas as gpd
import pandas as pd
import pysal

# load segments

In [2]:
def crs_prepossess(gpdf, bfr_crs, init_crs=4326):
    """
    create a shallow copy of gpdf; check the init crs of gpdf, if None, assign init_crs; change crs of copy to bfr_crs
    :param gpdf: geopandas.GeoDataFrame
    :param init_crs: init_crs epsg code
    :param bfr_crs: target crs epsg code used for buffering
    :return: a shallow copy of gpdf in bfr_crs
    """
    gpdf_crs = gpdf.copy()
    if gpdf_crs.crs == None:
        gpdf_crs.crs = {'init': u'epsg:{}'.format(init_crs)}
    return gpdf_crs.to_crs(epsg=bfr_crs)

In [ ]:
# PySAL compute euclidean distance between centroid of shapes
# Therefore the segments should be represented in suitable CRS instead of lat/lon (4326)

seg_in_lat_lon = gpd.read_file('some/file/with/geometry/column')
seg_in_ph_crs = crs_prepossess(seg_in_lat_lon, crs_for_ph)

# compute weights matrix between segments

In [ ]:
# There are different kinds of distance weights
# e.g. K nearest neighbours
ks = [1,2,3,4,5,6,7,8, 9, 10]
ws_knn = {k: pysal.weights.KNN.from_dataframe(seg_in_ph_crs[['geometry']], k=k) for k in ks}

# e.g. distanct band
# There are 2 kinds of distant band weight, binary or non-binary. I don't know the exact difference between them. 
# with correct CRS, the unit of the following dbs is meter
dbs = [10, 50, 100, 150, 200, 300, 400]
ws_db_notbinary = {db: pysal.weights.DistanceBand.from_dataframe(seg_in_ph_crs[['geometry']], threshold=db, binary=False, silent=True) for db in dbs}
ws_db_binary = {db: pysal.weights.DistanceBand.from_dataframe(seg_in_ph_crs[['geometry']], threshold=db, binary=True, silent=True) for db in dbs}

In [ ]:
# for KNN, every shape has k neighbors
# but for distance band, there are some islands which has no neighbors
# I choose 150 meter binary distance band weight in the cycling safety project considering the percentage of islands.
islands = [ws_db_binary[db].islands.__len__() for db in dbs]
pd.DataFrame(list(zip(dbs, islands)), columns=['band', 'num_islands']).set_index('band').T

# compute moran i

In [ ]:
def compute_moran_i(ws, param_list, x, pname):
    res = []

    for i, cname in enumerate(x):
        if i % 40 == 0:
            print('i=', i, cname)
            
        data = x[cname]
        for p in param_list:
            w = ws[p]
            mi = pysal.Moran(data, w, two_tailed=True, permutations=999)
            res.append({
                'column':cname, 
                pname:p,
                'I': mi.I, 
                'EI': mi.EI, 
                'p_norm': mi.p_norm * 2,
                'p_rand': mi.p_rand * 2,
                'z_norm': mi.z_norm,
                'z_rand': mi.z_rand,
            })
    print('done computing', pname)
    df = pd.DataFrame(res)
    return df[['column', pname, 'I', 'EI', 'p_norm', 'p_rand', 'z_norm', 'z_rand', 'p_sim', 'p_z_sim']]

In [ ]:
# make sure the index of X corresponds with the segment, starting from 0

X = pd.read_csv('the/feature/dataframe')

In [ ]:
# the function loop over each column of the features X. 
# It took quite a while for me to compute 200 features, maybe 1 hour or 2
# PySAL provides a function to compute moran I directly from a dataframe
# I don't know if that function would be faster

df_db_binary = compute_moran_i(ws_db_binary, dbs, x, 'db_b')

In [1]:
# p_norm and z_norm are the p and z value under normal distribution assumption
# p_rand and z_rand are the p and z value under random assumption
# I don't know what is the exact difference between them, but I chose p_rand<0.05 to identify significant features

def len_sig_features(df):
    return((df.p_rand<0.05).sum())

print(X.shape)
df_db_binary_total.groupby('db_b').apply(len_sig_features)

# average of neighbors' feature

In [ ]:
%matplotlib inline

In [ ]:
band = 150
db_150 = df_db_binary.db_b==band
sig_rand = df_db_binary.p_rand<0.05
sig_cols = df_db_binary[db_150 & sig_rand].column.tolist()

In [ ]:
df_db_binary[db_150 & sig_rand].I.plot(kind='hist',title='Moran I distribution')

In [ ]:
# the neighbors of the first segment
ws_db_binary[150].neighbors[0]

In [ ]:
# this also takes quite a while to compute
# maybe you can find a better way

res = {}
for col in sig_cols:
    column = x[col]
    new_col = {}
    for i, neighbor in ws_db_binary[band].neighbors.items():
        new_col[i] = column[neighbor].mean()
        res[col+'_neighbor'] = new_col


In [ ]:
x_neighbor = pd.DataFrame(res)

# you may want to store the result first
x_neighbor.to_csv('some/csv/file')

In [ ]:
# positive Moran I means spatially clustered effect, negative means spatially dispersed effect
# the larger the abs(I) is, the stronger the effect is
# You may wanna keep features with strong effect only

for i_thres in [0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9]:
    pass_i_thres = df_db_binary.I.abs()>=i_thres
    keep_cols = df_db_binary[db_150 & sig_rand & pass_i_thres].column
    keep_cols = (keep_cols+'_neighbor').tolist()
#     x_neighbor[keep_cols].to_csv('spatial-corr/x-neighbor-db-150-binary-i-%0.1f.csv' % i_thres)
    print(i_thres, len(keep_cols))